# 3주차 Day3 — RAG 파이프라인 실습

> **오후 실습용** | 수강생 직접 실행 노트북
>
> 오전에 데모로 본 RAG 시스템을 **직접 만들어봅니다.**

---

## 실습 구성

| 블록 | Section | 내용 | 시간 |
|------|---------|------|------|
| **오후1** (13:30~14:20) | Section 0~2 | 환경설정 + 벡터DB + 검색 테스트 | 10분 |
| | **Section 3** | **RAG 체인 구축 + 테스트 ** | 25분 |
| | 체크포인트 | 4개 항목 통과 확인 | 10분 |
| **오후2** (14:30~15:20) | Section 4 | 커스텀 프롬프트 적용 | 20분 |
| | Section 5 | 출처 표시 함수 구현 | 20분 |
| | Section 6 | 연습 과제 | 10분 |

---

### 학습 목표
1. PDF 문서 기반 RAG 파이프라인을 end-to-end로 구축할 수 있다
2. 출처(파일명 + 페이지)를 포함한 신뢰할 수 있는 응답을 생성할 수 있다
3. 커스텀 프롬프트로 답변 품질을 개선할 수 있다

---
---
# Section 0: 환경설정

> **2분** | 점심에 미리 실행하셨다면 바로 다음 섹션으로!
>
> 이 셀을 실행하면 필요한 패키지가 설치됩니다.

In [ ]:
# ============================================================
# Section 0: 패키지 설치
# ============================================================


import os  # 환경 변수(시스템 설정값)를 다루는 파이썬 기본 라이브러리


# -----------------------------------------------------------
# TensorFlow + protobuf 충돌 방지
# -----------------------------------------------------------
# HuggingFace 임베딩 모델은 PyTorch를 사용합니다.
# 같은 환경에 TensorFlow가 있으면 protobuf 라이브러리 버전 충돌로 오류가 납니다.
# 반드시 다른 라이브러리 import 보다 먼저 실행해야 합니다!


os.environ["USE_TF"] = "0"                           # TensorFlow 비활성화 (충돌 방지)
os.environ["USE_TORCH"] = "1"                        # PyTorch 활성화 (임베딩 모델에 필요)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"             # TF 관련 경고 메시지 숨기기
os.environ["TOKENIZERS_PARALLELISM"] = "false"       # 토크나이저 병렬 처리 경고 억제
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"  # Windows 심링크 경고 숨기기


# -----------------------------------------------------------
# HuggingFace 다운로드 서버 우회 (방화벽 환경 대응)
# -----------------------------------------------------------
# 회사/학교 네트워크에서 huggingface.co가 차단된 경우
# hf-mirror.com 미러 서버로 우회합니다.


os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"


# -----------------------------------------------------------
# 필요한 패키지 설치
# -----------------------------------------------------------
# !pip install: 외부 라이브러리를 인터넷에서 설치하는 명령 (-q: 로그 최소화)


!pip install -q langchain langchain-community langchain-huggingface langchain-text-splitters chromadb pypdf sentence-transformers


# 설치되는 패키지:
#   langchain               : AI 부품들을 파이프라인으로 연결하는 프레임워크
#   langchain-community     : PDF 로더, ChromaDB 연결 등 확장 도구
#   langchain-huggingface   : HuggingFace 임베딩을 LangChain에 연결하는 어댑터
#   langchain-text-splitters: 문서를 일정 크기로 잘라주는 청킹 도구
#   chromadb                : 벡터를 저장하고 유사도 검색하는 벡터 데이터베이스
#   pypdf                   : PDF 파일에서 텍스트를 추출하는 라이브러리
#   sentence-transformers   : 문장을 384차원 숫자 벡터로 변환하는 경량 AI 모델 (약 90MB)

In [ ]:
# ============================================================
# API 키 설정 + Gemini LLM 연결
# Google Gemini API 키를 입력하세요.
# aistudio.google.com -> Get API key 에서 무료 발급 가능
# ============================================================


import os
from pathlib import Path              # 파일 경로 처리
from google import genai as google_genai  # Google Gemini AI SDK


# API 키 입력 (아래 문자열을 본인의 API 키로 교체하세요)


os.environ["GOOGLE_API_KEY"] = "AIzaSy...여기에_새_API_키_입력"  # <- 여기에 제공받은 키 입력


# 입력값 확인: placeholder 그대로면 오류를 냅니다


assert os.environ.get("GOOGLE_API_KEY") != "AIza...", "API 키를 입력해주세요!"


# -----------------------------------------------------------
# Gemini 클라이언트 생성
# -----------------------------------------------------------
# google_genai.Client: Gemini API 서버와 통신하는 클라이언트 객체
# api_key: 내 계정임을 인증하는 API 키


gemini_client = google_genai.Client(api_key=os.environ["GOOGLE_API_KEY"])


# -----------------------------------------------------------
# 사용 가능한 Gemini 모델 자동 선택
# -----------------------------------------------------------
# 계정 등급/지역에 따라 사용 가능한 모델이 다릅니다.
# 아래 후보를 순서대로 시도해서 처음으로 성공하는 모델을 사용합니다.
#   gemini-2.0-flash-lite : 빠르고 가벼움, 무료 티어 분당 15회
#   gemini-2.0-flash-001  : 중간 성능
#   gemini-2.5-flash      : 가장 강력, 무료 티어 분당 5회 (너무 빠르게 호출하면 오류)


GEMINI_MODEL = None
for candidate in ["gemini-2.0-flash-lite", "gemini-2.0-flash-001", "gemini-2.5-flash"]:
    try:
        gemini_client.models.generate_content(model=candidate, contents="test")  # 테스트 호출
        GEMINI_MODEL = candidate                              # 성공하면 이 모델 사용
        break                                                 # 첫 번째 성공 후 중단
    except Exception:
        pass                                                  # 실패하면 다음 모델 시도
assert GEMINI_MODEL, "사용 가능한 Gemini 모델이 없습니다."
print(f"API 키 설정 완료 (모델: {GEMINI_MODEL})")


# -----------------------------------------------------------
# PDF 위치 자동 탐색
# -----------------------------------------------------------
# 노트북 실행 위치에 상관없이 PDF 파일이 있는 폴더를 찾아 작업 디렉토리를 변경합니다.
# os.walk(): 현재 폴더 아래의 모든 하위 폴더를 재귀적으로 순회


pdf_name = "제품사양서_스마트냉장고_RF9000.pdf"
for root, dirs, files in os.walk(str(Path.cwd())):
    if pdf_name in files:
        os.chdir(root)                         # PDF가 있는 폴더로 작업 디렉토리 변경
        break
print("작업 디렉토리 설정 완료")

---
# Section 1: 벡터DB 구축

> **3분** | Day2에서 이미 해본 내용입니다. 셀 3개를 순서대로 실행하세요.
>
> 결과 숫자만 확인하면 됩니다.

In [ ]:
# ============================================================
# Step 1: PDF 로딩
# 사양서(4페이지) + 시험성적서(3페이지) = 총 7페이지를 읽어옵니다
# ============================================================

# PyPDFLoader: PDF 파일을 열어서 페이지별로 텍스트를 추출하는 도구
# 각 페이지는 Document 객체로 변환됩니다.
#   Document.page_content : 페이지의 텍스트 내용 (실제 글자들)
#   Document.metadata     : 파일 정보 {"source": "파일명.pdf", "page": 0}
#                           (주의: page는 0부터 시작 -> 출력 시 +1 해야 1페이지)


from langchain_community.document_loaders import PyPDFLoader


# 읽어올 PDF 파일 이름 목록 (cell-3에서 작업 디렉토리를 이미 PDF 폴더로 변경했습니다)


pdf_files = [
    "제품사양서_스마트냉장고_RF9000.pdf",   # 제품 스펙: 용량, 소비전력, 에러코드 등
    "시험성적서_스마트냉장고_RF9000.pdf"    # 시험 결과: 에너지효율, 소음, 안전시험 등
]

all_docs = []                           # 모든 페이지 Document를 모아둘 빈 리스트
for pdf_path in pdf_files:              # PDF 파일을 하나씩 순회
    loader = PyPDFLoader(pdf_path)      # PDF 읽기 도구 생성
    docs = loader.load()                # PDF를 읽어서 페이지별 Document 리스트 반환
    all_docs.extend(docs)               # extend: 리스트를 풀어서 이어붙임 (append와 달리 중첩 안 됨)
    print(f"{pdf_path.split('/')[-1]} -> {len(docs)}페이지 로딩")

print(f"\n총 {len(all_docs)}페이지 로딩 완료")  # <- 7이 나와야 합니다

In [ ]:
# ============================================================
# Step 2: 청킹 (문서를 작은 조각으로 분할)
# 한 페이지가 너무 길면 AI가 핵심을 못 찾습니다.
# 500자씩 잘라서 검색 정확도를 높이는 과정입니다.
# ============================================================


from langchain_text_splitters import RecursiveCharacterTextSplitter  # <- langchain_text_splitters


# RecursiveCharacterTextSplitter: 문단 -> 문장 -> 단어 순으로 자연스럽게 분할
# chunk_size=500   : 한 조각의 최대 글자 수 (A4 반 페이지 분량)
# chunk_overlap=50 : 인접 조각 사이에 50자를 겹치게 함
#                    예) 조각1의 마지막 50자 == 조각2의 첫 50자
#                    -> 경계에서 문맥이 끊기지 않도록 연결고리 역할


splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,                          # 한 조각의 최대 글자 수 (A4 반 페이지 정도)
    chunk_overlap=50                         # 조각 간 50자 겹침 (문맥이 끊기지 않도록)
)
chunks = splitter.split_documents(all_docs)  # 7페이지 -> 여러 조각으로 분할

print(f"{len(chunks)}개 청크로 분할")          # <- 10~20개 범위면 정상

In [ ]:
# ============================================================
# Step 3: 벡터DB 저장
# 각 조각을 숫자 벡터로 변환(임베딩)해서 ChromaDB에 저장합니다.
# HuggingFace 로컬 모델 사용 (인터넷 없이 내 컴퓨터에서 동작)
# ============================================================

from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings  # 로컬 임베딩 모델


# 임베딩(Embedding)이란?
#   텍스트를 수백 개의 숫자 배열(벡터)로 변환하는 과정입니다.
#   의미가 비슷한 문장은 비슷한 벡터를 가집니다.
#   -> "냉장고 용량"과 "냉장고 크기"는 가까운 벡터 (유사)
#   -> "냉장고 용량"과 "오늘 날씨"는 먼 벡터 (무관)
#   이를 이용해 질문과 가장 관련 있는 조각을 빠르게 찾을 수 있습니다.
#
# sentence-transformers/all-MiniLM-L6-v2:
#   - 경량 임베딩 모델 (크기 약 90MB, 384차원 벡터 생성)
#   - 한국어+영어 혼재 문서도 처리 가능
# device="cpu": GPU 없이 CPU만으로 실행 (느리지만 어디서나 동작)


embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)


# Chroma.from_documents: 각 청크를 임베딩 벡터로 변환한 뒤 DB에 일괄 저장
# 이후 "질문과 가장 유사한 조각 찾기"를 매우 빠르게 수행할 수 있게 됩니다


vectorstore = Chroma.from_documents(
    chunks,                                  # 위에서 만든 문서 조각들
    embeddings                               # 텍스트 -> 벡터 변환기
)

print(f"벡터DB 구축 완료 ({len(chunks)}개 청크 저장)")

---
# Section 2: 유사도 검색 테스트

> **2분** | Day2에서 배운 유사도 검색이 잘 되는지 빠르게 확인합니다.
>
> 벡터DB가 정상 구축됐는지 검증하는 단계입니다.

In [ ]:
# ============================================================
# Section 2: 유사도 검색 테스트 (Day2 복습)
# 벡터DB에서 질문과 의미가 비슷한 문서 조각 3개를 찾아봅니다.
# 이 검색이 RAG의 핵심 단계입니다. (5단계 중 Step 1~2)
# ============================================================


query = "RF9000의 총 용량은?"  # 테스트 질문


# similarity_search(query, k=3) 작동 방식:
#   1. query 텍스트를 임베딩(숫자 벡터)으로 변환
#   2. 벡터DB 안의 모든 청크와 코사인 유사도를 계산
#   3. 유사도가 높은 상위 k=3개의 청크를 반환


results = vectorstore.similarity_search(query, k=3)      # 가장 유사한 3개 검색

print(f"검색 결과: {len(results)}개 문서 조각 찾음")
print()
for i, doc in enumerate(results):                        # enumerate: 순번(i)과 내용(doc)을 함께 꺼냄
    # 메타데이터에서 파일명과 페이지 번호 추출
    src = doc.metadata.get('source', '').split('/')[-1]  # 경로 중 파일명만 가져옴
    pg = doc.metadata.get('page', 0) + 1                 # 0-based -> 1-based 변환
    print(f"[{i+1}] {src} (p.{pg})")
    print(f"  {doc.page_content[:80]}...")               # 조각 내용 앞 80자만 미리보기
    print()

> 검색 결과가 3개 나오고, 제품사양서의 용량 관련 내용이 보이면 정상입니다.
>
> 이 검색 결과가 다음 Section 3에서 RAG 함수의 **similarity_search**가 자동으로 가져오는 것과 같습니다.

---
---
# Section 3: RAG 체인 구축

> **25분** | **오늘의 핵심!** 오전2에서 배운 코드를 직접 타이핑합니다.
>
> 오전에 슬라이드에서 본 5줄 코드, 기억나시죠?
>
> 이제 직접 만들어봅시다.


In [ ]:
# ============================================================
# RAG 질의 함수 - 오늘의 핵심 코드!
#
# 아래 5단계를 하나의 함수로 묶어서 실행합니다:
#   Step 1. 질문을 임베딩(숫자 벡터)으로 변환
#   Step 2. 벡터DB에서 유사 문서 조각 3개 검색
#   Step 3. 검색된 조각들을 하나의 컨텍스트 텍스트로 합치기
#   Step 4. [컨텍스트 + 질문]을 Gemini LLM에 전달 -> 답변 생성
#   Step 5. 답변 + 출처 정보를 딕셔너리로 반환
# ============================================================


def qa_invoke(query):    # query: 사용자가 입력한 질문 문자열
                         # [Step 1~2] 질문과 의미가 비슷한 문서 조각 3개 검색
                         # similarity_search: 질문을 벡터로 바꾼 뒤 DB에서 유사도 높은 조각 반환
   
    docs = vectorstore.similarity_search(query, k=3)


    # [Step 3] 검색된 조각 3개를 하나의 긴 텍스트(컨텍스트)로 합치기
    # "\n\n".join(): 각 조각 사이에 빈 줄을 넣어서 이어붙임


    context = "\n\n".join(doc.page_content for doc in docs)


    # [Step 4] 프롬프트 완성 후 Gemini에 전송 -> 답변 텍스트 수신
    # f-string으로 {context}와 {query} 자리에 실제 값을 채워 완성된 지시문 생성


    prompt = f"다음 문서를 참고하여 질문에 답하세요.\n\n문서 내용:\n{context}\n\n질문: {query}\n\n답변:"
    answer = gemini_client.models.generate_content(
        model=GEMINI_MODEL,                                  # 앞서 자동 선택된 Gemini 모델
        contents=prompt                                      # 완성된 프롬프트 전송
    ).text                                                   # .text: 응답 객체에서 텍스트만 추출
         
    source_documents = docs                                  # [Step 5] 출처 정보 보존 (docs 리스트에 들어있는 메타데이터 활용)

                                                             # 딕셔너리 형태로 반환
                                                             #   result["result"]           -> 답변 텍스트
                                                             #   result["source_documents"] -> 출처 Document 리스트
    return {"result": answer, "source_documents": source_documents}

print("qa_invoke() 함수 생성 완료!")
print('사용법: result = qa_invoke("질문")')
print('답변 출력: print(result["result"])')
print('출처 접근: result["source_documents"]')

### 첫 질의 실행!

> 코드를 실행하고 첫 번째 질문을 던져보세요.

In [ ]:
# ============================================================
# 첫 번째 질문!
# ============================================================


result = qa_invoke("RF9000의 총 용량은?")  # 질문 실행!

print("답변:")
print(result["result"])

print("\n출처:")
for doc in result["source_documents"]:
    src = doc.metadata.get("source", "").split("/")[-1]
    pg = doc.metadata.get("page", 0) + 1
    print(f"- {src} (p.{pg})")

답변:
제공된 문서에는 RF9000의 총 용량 정보가 명시되어 있지 않습니다.

출처:
- 시험성적서_스마트냉장고_RF9000.pdf (p.1)
- 제품사양서_스마트냉장고_RF9000.pdf (p.1)
- 제품사양서_스마트냉장고_RF9000.pdf (p.3)


> **답변이 나왔다면, 축하합니다! 여러분의 RAG가 동작합니다!**
>
> 예상 결과: RF9000의 총 용량은 868L (냉장 524L / 냉동 344L)
>
> 출처: 제품사양서_스마트냉장고_RF9000.pdf (p.2)

### 질의 5개 테스트

> 아래 질문을 하나씩 실행하고 답변을 확인하세요.
>
> 각 질문마다 확인할 것:
> 1. 답변이 정확한가?
> 2. 출처(source_documents)가 나오는가?
> 3. 어떤 PDF의 몇 페이지인가?

In [ ]:
# Q1: 사양서 질문

result = qa_invoke("RF9000의 총 용량은?")       # 질문 실행
print("Q1:", result["result"])                 # 답변 출력
for doc in result["source_documents"]:         # 출처 순회
    print(f"  출처: {doc.metadata.get('source','').split('/')[-1]} (p.{doc.metadata.get('page',0)+1})")

Q1: 제공된 문서에는 RF9000의 총 용량에 대한 정보가 없습니다.
  출처: 시험성적서_스마트냉장고_RF9000.pdf (p.1)
  출처: 제품사양서_스마트냉장고_RF9000.pdf (p.1)
  출처: 제품사양서_스마트냉장고_RF9000.pdf (p.3)


In [ ]:
# Q2: 사양서 질문 — 사양서 p.2에 답이 있습니다

result = qa_invoke("소비전력은 얼마인가요?") 
print("Q2:", result["result"])  
for doc in result["source_documents"]: 
    print(f"  출처: {doc.metadata.get('source','').split('/')[-1]} (p.{doc.metadata.get('page',0)+1})")

Q2: 문서에 따르면 소비전력은 다음과 같습니다:

*   **정격 소비전력:** 36 W
*   **월간 소비전력량:** 28.5 kWh/월
  출처: 제품사양서_스마트냉장고_RF9000.pdf (p.4)
  출처: 제품사양서_스마트냉장고_RF9000.pdf (p.3)
  출처: 제품사양서_스마트냉장고_RF9000.pdf (p.2)


In [ ]:
# Q3: 사양서 질문 — 사양서 p.4 에러코드표에 답이 있습니다

result = qa_invoke("E3 에러코드는 무엇인가요?")  
print("Q3:", result["result"])  
for doc in result["source_documents"]:  
    print(f"  출처: {doc.metadata.get('source','').split('/')[-1]} (p.{doc.metadata.get('page',0)+1})")

In [ ]:
# Q4: 시험성적서 질문 — 사양서가 아닌 시험성적서에서 답이 나와야 합니다

result = qa_invoke("에너지효율 등급은?") 
print("Q4:", result["result"]) 
for doc in result["source_documents"]: 
    print(f"  출처: {doc.metadata.get('source','').split('/')[-1]} (p.{doc.metadata.get('page',0)+1})")

In [ ]:
# Q5: 시험성적서 질문 — 시험성적서 p.2 소음 시험 항목에 답이 있습니다

result = qa_invoke("소음 측정값은 얼마인가요?")  
print("Q5:", result["result"])  
for doc in result["source_documents"]:  
    print(f"  출처: {doc.metadata.get('source','').split('/')[-1]} (p.{doc.metadata.get('page',0)+1})")

### 오후1 체크포인트 — 4개 항목 통과하기

| # | 항목 | 확인 |
|---|------|------|
| 1 | 패키지 설치 완료 (`pip install` 에러 없이 완료) | |
| 2 | `all_docs = 7개`, `chunks ≈ 15~20개` | |
| 3 | RAG 답변 + `source_documents` 출력 | |
| 4 | 질의 3개 이상 테스트 완료 (사양서 2개 + 시험성적서 1개 이상) | |

> 4개 다 되셨으면 손 들어주세요!
>
> 쉬는 시간 후 **오후2**: 커스텀 프롬프트 + 출처 표시를 추가합니다.

---
---
# Section 4: 커스텀 프롬프트 — 답변 규칙 추가

> **20분** | 오전 브레인스토밍에서 만든 규칙, 기억나시죠?
>
> 지금 그 규칙을 코드에 직접 넣어봅니다!
>
> **기본 체인 vs 커스텀 체인**의 답변 차이를 직접 확인하세요.

In [ ]:
# ============================================================
# Section 4: 커스텀 프롬프트 - 답변 규칙 추가
#
# 2주차에 배운 PromptTemplate을 RAG에 적용합니다.
# {context}  = 벡터DB에서 검색된 문서 조각 (자동으로 채워짐)
# {question} = 사용자가 입력한 질문 (자동으로 채워짐)
# ============================================================


from langchain_core.prompts import PromptTemplate         # <- langchain_core

# PromptTemplate이란?
#   LLM에게 보낼 지시문의 틀(템플릿)입니다.
#   {변수명} 자리에 실제 값을 채워 넣으면 완성된 프롬프트가 됩니다.
#   input_variables 목록에 있는 변수만 format()으로 채울 수 있습니다.
#
# 규칙을 추가할수록 AI가 원하는 방향으로 답변합니다.
#   예) "단위 포함" 규칙  -> 수치 답변 시 L, W, dB 등 단위 자동 포함
#       "모르면 모른다고" -> 할루시네이션(AI가 없는 정보를 지어내는 현상) 방지


custom_prompt = PromptTemplate(
    template="""다음 문서를 참고하여 질문에 답하세요.

문서 내용: {context}
질문: {question}

답변 규칙:
1. 문서에 있는 정보만 사용하세요
2. 문서에 없으면 "해당 정보를 찾을 수 없습니다"라고 답하세요
3. 수치를 인용할 때는 단위를 반드시 포함하세요

답변:""",
    input_variables=["context", "question"]          # 템플릿 안에서 채워질 변수 이름 목록
)

print("커스텀 프롬프트 생성 완료")
print("\n답변 규칙 3가지:")
print("1. 문서에 있는 정보만 사용")
print("2. 없으면 '찾을 수 없습니다'")
print("3. 수치에 단위 포함")

In [ ]:
# ============================================================
# 커스텀 프롬프트를 적용한 RAG 함수
# qa_invoke()와 구조는 같지만 커스텀 프롬프트(규칙 3가지)를 사용합니다.
# ============================================================


def custom_qa_invoke(query):                                           # query: 사용자 질문 문자열
    docs = vectorstore.similarity_search(query, k=3)                   # [Step 1~2] 질문과 유사한 문서 조각 3개 검색
    
    context = "\n\n".join(doc.page_content for doc in docs)            # [Step 3] 검색된 조각들을 하나의 컨텍스트 텍스트로 합치기

                                                                       # [Step 4] 커스텀 프롬프트에 context와 question 값을 채워서 Gemini에 전송                                                                       
    prompt = custom_prompt.format(context=context, question=query)     # .format(): {context}와 {question} 자리에 실제 값 삽입 -> 완성된 프롬프트 생성
    answer = gemini_client.models.generate_content(
        model=GEMINI_MODEL,
        contents=prompt
    ).text                                                             # .text: 응답 객체에서 텍스트만 추출

    return {"result": answer, "source_documents": docs}

print("커스텀 RAG 함수 생성 완료")
print("사용법: result = custom_qa_invoke('질문')")

### 기본 vs 커스텀 비교 테스트

> 같은 질문을 `qa` (기본)과 `custom_qa` (커스텀)에 각각 던져보세요.
>
> 차이가 보이면 성공!

In [ ]:
# 같은 질문으로 기본 vs 커스텀 비교!

query = "절연저항 시험 결과는?"

print("=" * 50)
print("[기본 체인]")
print("=" * 50)
r1 = qa_invoke(query)          # 규칙 없는 기본 체인
print(r1["result"])

print()
print("=" * 50)
print("[커스텀 체인]")
print("=" * 50)
r2 = custom_qa_invoke(query)   # 규칙 3가지가 적용된 커스텀 체인
print(r2["result"])

[기본 체인]
제공된 문서에는 절연저항 시험 결과에 대한 정보가 없습니다.

[커스텀 체인]
해당 정보를 찾을 수 없습니다.


In [ ]:
# 문서에 없는 질문으로 비교! — 할루시네이션 방지 테스트

query = "RF9000 가격은 얼마인가요?"             # 가격은 사양서에도 시험성적서에도 없음

print("=" * 50)
print("[기본 체인] — 가격을 지어낼 수 있음!")
print("=" * 50)
r1 = qa_invoke(query)                         # 규칙 없음 → 답변을 지어낼 수 있음
print(r1["result"])

print()
print("=" * 50)
print("[커스텀 체인] — '찾을 수 없습니다'로 답해야 정상")
print("=" * 50)
r2 = custom_qa_invoke(query)   # 규칙 2번 → 없으면 "찾을 수 없습니다"
print(r2["result"])

[기본 체인] — 가격을 지어낼 수 있음!
제공된 문서에는 RF9000 제품의 가격 정보가 나와 있지 않습니다.

[커스텀 체인] — '찾을 수 없습니다'로 답해야 정상
해당 정보를 찾을 수 없습니다.


> **차이가 보이셨나요?**
>
> - 기본 체인: 수치나 단위가 빠질 수 있고, 없는 정보를 지어낼 수 있음
> - 커스텀 체인: 규칙 덕분에 정확한 수치+단위, 모르면 "찾을 수 없다"고 답변
>
> **프롬프트 규칙 하나가 이만큼 차이를 만듭니다!**

### 여러분의 규칙 추가!

> 오전 브레인스토밍에서 만든 규칙을 4번에 추가해보세요.
>
> 예시:
> - `4. 표 형식으로 정리해 답변`
> - `4. 안전 수치는 기준값도 포함`
> - `4. 출처 문서명을 답변에 포함`

In [ ]:
# ============================================================
# 여러분의 규칙을 추가해보세요!
# ============================================================

# my_prompt: custom_prompt에 4번째 규칙을 추가한 나만의 프롬프트
# [여기에 규칙 추가] 부분을 원하는 규칙으로 바꿔보세요.
# 예시:
#   4. 표 형식으로 정리해서 답변하세요
#   4. 답변 마지막에 "출처: [파일명] p.[페이지]"를 반드시 붙이세요
#   4. 100자 이내로 간결하게 답변하세요


my_prompt = PromptTemplate(
    template="""다음 문서를 참고하여 질문에 답하세요.

문서 내용: {context}
질문: {question}

답변 규칙:
1. 문서에 있는 정보만 사용하세요
2. 문서에 없으면 "해당 정보를 찾을 수 없습니다"라고 답하세요
3. 수치를 인용할 때는 단위를 반드시 포함하세요
4. [여기에 규칙 추가]

답변:""",
    input_variables=["context", "question"]
)

def my_qa_invoke(query):                                        # 나만의 규칙이 적용된 RAG 함수
    docs = vectorstore.similarity_search(query, k=3)
    context = "\n\n".join(doc.page_content for doc in docs)     # my_prompt.format(): {context}와 {question} 자리에 실제 값 삽입
    prompt = my_prompt.format(context=context, question=query)
    answer = gemini_client.models.generate_content(model=GEMINI_MODEL, contents=prompt).text
    return {"result": answer, "source_documents": docs}


# 테스트 - 내 규칙이 잘 적용됐는지 확인


result = my_qa_invoke("E3 에러코드는 무엇인가요?")
print(result["result"])

---
---
# Section 5: 출처 표시 함수 구현

> **20분** | 오전3에서 본 `format_response_with_sources()` 함수를 직접 만듭니다.
>
> 질문 + 답변 + 출처를 한 번에 보여주는 함수입니다.

In [ ]:
# ============================================================
# Section 5: 출처 표시 함수
# ============================================================

def ask_with_sources(query):


    # 질문하고, 답변과 출처를 함께 보기 좋게 출력하는 함수
    # custom_qa_invoke()로 RAG를 실행한 뒤 결과를 정리해서 출력합니다.

    # custom_qa_invoke(): 커스텀 프롬프트를 적용한 RAG 실행
    # 반환값: {"result": 답변텍스트, "source_documents": [Document, ...]}


    result = custom_qa_invoke(query)

    # 결과 출력

    print("=" * 50)
    print(f"질문: {query}")
    print("=" * 50)
    print(f"\n답변:\n{result['result']}")


    # 출처 목록 출력
    # result['source_documents']: 검색에 사용된 Document 객체들의 리스트
    # doc.metadata: 각 Document의 메타데이터 (파일명, 페이지 번호 등)


    print(f"\n참고 문서 ({len(result['source_documents'])}건):")
    for i, doc in enumerate(result['source_documents']):
        name = doc.metadata.get('source', '').split('/')[-1]   # 경로 중 파일명만 추출
        pg = doc.metadata.get('page', 0) + 1                   # 0-based -> 1-based
        print(f"  [{i+1}] {name} (p.{pg})")
    print("=" * 50)

print("ask_with_sources() 함수 정의 완료")
print('\n사용법: ask_with_sources("질문")')
print("-> 답변 + 출처 파일명 + 페이지 번호를 함께 출력합니다")

In [ ]:
# 테스트: 사양서 질문

ask_with_sources("RF9000 냉매 종류는?")  # 함수 하나로 답변 + 출처 한 번에 출력


질문: RF9000 냉매 종류는?

답변:
RF9000 냉매 종류는 R-600a (이소부탄)입니다.

참고 문서 (3건):
  [1] 시험성적서_스마트냉장고_RF9000.pdf (p.1)
  [2] 제품사양서_스마트냉장고_RF9000.pdf (p.1)
  [3] 제품사양서_스마트냉장고_RF9000.pdf (p.2)


In [ ]:
# 테스트: 시험성적서 질문 — 출처가 시험성적서로 나오는지 확인

ask_with_sources("에너지효율 등급과 월간 소비전력량은?")


질문: 에너지효율 등급과 월간 소비전력량은?

답변:
에너지소비효율 1등급, 월간 소비전력량 28.5 kWh/월

참고 문서 (3건):
  [1] 제품사양서_스마트냉장고_RF9000.pdf (p.4)
  [2] 제품사양서_스마트냉장고_RF9000.pdf (p.4)
  [3] 제품사양서_스마트냉장고_RF9000.pdf (p.2)


In [ ]:
# 테스트: 교차 참조 — 두 문서에서 답변이 나오는지 확인!
# 출처에 사양서와 시험성적서가 함께 나오면 성공

ask_with_sources("이 제품은 안전한가요?")


질문: 이 제품은 안전한가요?

답변:
해당 정보를 찾을 수 없습니다.

참고 문서 (3건):
  [1] 제품사양서_스마트냉장고_RF9000.pdf (p.4)
  [2] 제품사양서_스마트냉장고_RF9000.pdf (p.4)
  [3] 제품사양서_스마트냉장고_RF9000.pdf (p.2)


> **출처에 두 PDF가 함께 나오면 교차 참조가 된 것입니다!**
>
> 출처를 실제 PDF와 대조해보세요. 페이지 번호가 맞나요?

In [ ]:
# 테스트: 문서에 없는 질문 — "찾을 수 없습니다"로 답해야 정상

ask_with_sources("RF9000 가격은 얼마인가요?")


질문: RF9000 가격은 얼마인가요?

답변:
해당 정보를 찾을 수 없습니다.

참고 문서 (3건):
  [1] 시험성적서_스마트냉장고_RF9000.pdf (p.1)
  [2] 제품사양서_스마트냉장고_RF9000.pdf (p.2)
  [3] 제품사양서_스마트냉장고_RF9000.pdf (p.1)


> 가격은 사양서에도 시험성적서에도 없습니다.
>
> 커스텀 프롬프트 규칙 2번 덕분에 "찾을 수 없습니다"라고 답합니다.
>
> 이것이 **할루시네이션 방지** — 내일 Day4에서 더 깊이 다룹니다.

---
---
# Section 6: 연습 과제

> **10분** | 다 못 끝내도 괜찮습니다. 핵심은 Section 5까지 완료하는 것!

---

## 기본 과제: 질문 5개 + 출처 대조

In [ ]:
# 기본 과제 Q1: 사양서 질문 — 사양서 p.2 주요 사양표에서 확인

ask_with_sources("냉장실 용량은 몇 리터?")


질문: 냉장실 용량은 몇 리터?

답변:
해당 정보를 찾을 수 없습니다.

참고 문서 (3건):
  [1] 제품사양서_스마트냉장고_RF9000.pdf (p.4)
  [2] 제품사양서_스마트냉장고_RF9000.pdf (p.4)
  [3] 시험성적서_스마트냉장고_RF9000.pdf (p.1)


In [ ]:
# 기본 과제 Q2: 사양서 질문 — 사양서 p.4 에러코드 일람에서 확인

ask_with_sources("E3 에러코드는 무엇인가요?")


In [ ]:

# 기본 과제 Q3: 시험성적서 질문 — 시험성적서 p.2 전기안전시험에서 확인
ask_with_sources("절연저항 시험 합격 기준은?")


In [ ]:
# 기본 과제 Q4: 시험성적서 질문 — 시험성적서 p.2 소음 시험에서 확인

ask_with_sources("소음 측정값은?")


질문: 소음 측정값은?

답변:
해당 정보를 찾을 수 없습니다

참고 문서 (3건):
  [1] 제품사양서_스마트냉장고_RF9000.pdf (p.3)
  [2] 제품사양서_스마트냉장고_RF9000.pdf (p.4)
  [3] 제품사양서_스마트냉장고_RF9000.pdf (p.2)


In [ ]:
# 기본 과제 Q5: 교차 참조 — 사양서 + 시험성적서 모두 출처에 나오는지 확인

ask_with_sources("이 제품은 안전한가요?")


## 심화 과제: "문서에 없는 질문" 3개

> 커스텀 프롬프트 덕분에 "해당 정보를 찾을 수 없습니다"라고 답해야 정상!
>
> → 잘못된 출처를 발견하면 메모해두세요! 내일(Day4) 할루시네이션 대응에서 다시 다룹니다.

In [ ]:
# 심화 Q1: 가격 — 사양서/시험성적서 어디에도 없음 → "찾을 수 없습니다"

ask_with_sources("RF9000 가격은 얼마인가요?")


In [ ]:
# 심화 Q2: 경쟁사 비교 — 다른 회사 제품 정보는 문서에 없음

ask_with_sources("삼성 냉장고와 비교하면?")


In [ ]:
# 심화 Q3: AS 전화번호 — 고객 서비스 정보는 문서에 없음

ask_with_sources("이 제품 AS 전화번호는?")


질문: 이 제품 AS 전화번호는?

답변:
해당 정보를 찾을 수 없습니다.

참고 문서 (3건):
  [1] 제품사양서_스마트냉장고_RF9000.pdf (p.4)
  [2] 제품사양서_스마트냉장고_RF9000.pdf (p.3)
  [3] 제품사양서_스마트냉장고_RF9000.pdf (p.2)


---

## 자유 질문

> 궁금한 질문을 직접 만들어서 테스트해보세요!

In [ ]:
ask_with_sources("") # ← 여기에 질문 입력

In [ ]:
ask_with_sources("") # ← 여기에 질문 입력

In [ ]:
ask_with_sources("") # ← 여기에 질문 입력

---
---
# 실습 완료!

## 오늘 만든 것 정리

| 산출물 | 확인 |
|--------|------|
| 기본 RAG: `qa` 체인으로 문서에 질문 → 답변 + 출처 | |
| 커스텀 프롬프트: 규칙 3가지 + 나의 규칙 추가 | |
| 출처 표시: `ask_with_sources()` 함수 구현 | |
| "모름" 응답: 문서에 없으면 지어내지 않기 | |

---

### 오후3 (15:30~16:30): 미니퀴즈 + 조별 토론

> **이 노트북을 제출합니다.** (산출물 확인용)